In [1]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import pandas as pd
import numpy as np

# Setup headless Chrome
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=103819/Roger-Federer&table=mcp-serve'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='mcp-serve')
print(table.prettify())

driver.quit()


<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="mcp-serve">
 <thead>
  <tr>
   <th align="left">
    <span title="">
     Match
    </span>
   </th>
   <th align="left">
    <span title="">
     Result
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of serves that
were unreturned">
     Unret%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of service points
won on either the serve or second shot">
     &lt;=3 W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of points won when
return was put in play">
     RiP W%
    </span>
   </th>
   <th align="right" class="header">
    <span title="Serve Impact: Advanced stat
estimating how many service points
were won due to the serve">
     SvImpact
    </span>
   </th>
   <th align="right" class="header">
    <span title="Percent of first serves
that were unreturned">
     1st: Unret%
    </span>
   </th>
   <th a

In [2]:
data = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            data.append((cell.get_text(strip=True)))


2021 Wimbledon QF
L vsHurkacz
22.6%
43.0%
43.5%
34.1%
31.1%
52.5%
52.4%
48.7%
39.3%
54.5%
37.5%
6.9%
27.6%
29.6%
-49
2021 Wimbledon R16
W vsSonego
37.2%
52.6%
58.3%
44.0%
44.7%
59.6%
61.5%
56.0%
36.4%
40.0%
100.0%
26.7%
43.3%
54.5%
-30
2021 Wimbledon R32
W vsNorrie
38.6%
56.1%
52.9%
48.8%
39.5%
60.5%
56.5%
54.7%
59.5%
41.0%
0.0%
36.8%
47.4%
45.8%
-79
2021 Wimbledon R64
W vsGasquet
34.7%
54.7%
61.3%
45.1%
42.9%
68.3%
72.2%
58.4%
56.7%
48.5%
66.7%
18.8%
28.1%
46.2%
-79
2021 Wimbledon R128
W vsMannarino
29.5%
41.1%
57.9%
36.2%
41.2%
54.4%
55.0%
52.2%
37.1%
54.5%
100.0%
12.2%
22.0%
61.1%
7
2021 Roland Garros R32
W vsKoepfer
20.3%
35.4%
55.6%
28.6%
25.0%
43.0%
57.3%
38.2%
53.5%
38.6%
33.3%
12.5%
23.2%
53.1%
-53
2021 Roland Garros R64
W vsCilic
40.2%
54.7%
49.3%
47.7%
43.8%
61.3%
55.6%
54.8%
43.2%
58.1%
50.0%
33.3%
41.7%
37.5%
-38
2021 Roland Garros R128
W vsIstomin
35.4%
58.5%
69.0%
47.7%
45.7%
67.4%
64.0%
63.0%
52.6%
63.0%
-
10.5%
36.8%
76.5%
-79
2021 Doha R16
W vsEvans
23.8%
43.8%
61.3%
3

In [3]:
data = [d.replace('\xa0',' ') for d in data]

In [4]:
year_tourney_round = data[::17] # year tournament round in a single string

In [5]:
df_rows = []
for i in range(0, len(data)):
    if i % 17 == 0:
        row = data[i:i+17]
        df_rows.append(row)

In [6]:
cols = ['year tourney round', 'result opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%',
        '1st: <=3 W%', '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%', '2nd: Unret%', '2nd: <=3 W%',
        '2nd: RiP W%', '2ndAgg']
df = pd.DataFrame(df_rows, columns=cols)
df['year'] = df['year tourney round'].apply(lambda x: x.split(' ')[0])
df['tourney'] = df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
df['round'] = df['year tourney round'].apply(lambda x: x.split(' ')[-1]) 
df['result'] = df['result opponent'].apply(lambda x: x.split(' ')[0])
df['opponent'] = df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
df.head(10)

,overall: Unret%,overall: <=3 W%,overall: RiP W%,overall: SvImpact,1st: Unret%,1st: <=3 W%,1st: RiP W%,1st: SvImpact,1st: D Wide%,1st: A Wide%,1st: A BP Wide%,2nd: Unret%,2nd: <=3 W%,2nd: RiP W%,2ndAgg,year,tourney,round,result,opponent
0,22.6%,43.0%,43.5%,34.1%,31.1%,52.5%,52.4%,48.7%,39.3%,54.5%,37.5%,6.9%,27.6%,29.6%,-49,2021,Wimbledon,QF,L,Hurkacz
1,37.2%,52.6%,58.3%,44.0%,44.7%,59.6%,61.5%,56.0%,36.4%,40.0%,100.0%,26.7%,43.3%,54.5%,-30,2021,Wimbledon,R16,W,Sonego
2,38.6%,56.1%,52.9%,48.8%,39.5%,60.5%,56.5%,54.7%,59.5%,41.0%,0.0%,36.8%,47.4%,45.8%,-79,2021,Wimbledon,R32,W,Norrie
3,34.7%,54.7%,61.3%,45.1%,42.9%,68.3%,72.2%,58.4%,56.7%,48.5%,66.7%,18.8%,28.1%,46.2%,-79,2021,Wimbledon,R64,W,Gasquet
4,29.5%,41.1%,57.9%,36.2%,41.2%,54.4%,55.0%,52.2%,37.1%,54.5%,100.0%,12.2%,22.0%,61.1%,7,2021,Wimbledon,R128,W,Mannarino
5,20.3%,35.4%,55.6%,28.6%,25.0%,43.0%,57.3%,38.2%,53.5%,38.6%,33.3%,12.5%,23.2%,53.1%,-53,2021,Roland Garros,R32,W,Koepfer
6,40.2%,54.7%,49.3%,47.7%,43.8%,61.3%,55.6%,54.8%,43.2%,58.1%,50.0%,33.3%,41.7%,37.5%,-38,2021,Roland Garros,R64,W,Cilic
7,35.4%,58.5%,69.0%,47.7%,45.7%,67.4%,64.0%,63.0%,52.6%,63.0%,-,10.5%,36.8%,76.5%,-79,2021,Roland Garros,R128,W,Istomin
8,23.8%,43.8%,61.3%,38.2%,29.6%,53.5%,70.0%,50.8%,55.2%,50.0%,0.0%,11.8%,23.5%,46.7%,-79,2021,Doha,R16,W,Evans
9,28.8%,43.3%,42.3%,37.9%,35.3%,55.9%,47.7%,49.1%,65.7%,48.5%,33.3%,18.2%,21.2%,33.3%,47,2020,Australian Open,SF,L,Djokovic


In [7]:
neworder = ['year','tourney', 'round', 'result', 'opponent', 'overall: Unret%', 'overall: <=3 W%', 'overall: RiP W%', 'overall: SvImpact', '1st: Unret%', '1st: <=3 W%',
       '1st: RiP W%', '1st: SvImpact', '1st: D Wide%', '1st: A Wide%', '1st: A BP Wide%',
       '2nd: Unret%', '2nd: <=3 W%', '2nd: RiP W%', '2ndAgg']

df = df[neworder]

In [8]:

df.to_csv('../data/federer_serve_chart.csv') #save to csv in data folder

In [9]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
driver = webdriver.Chrome(options=options)

# Go to the page
url = 'https://www.tennisabstract.com/cgi-bin/player-more.cgi?p=103819/Roger-Federer&table=serve-speed'
driver.get(url)

# Wait for the page to load content (you can adjust this time or use WebDriverWait)
time.sleep(3)

# Get the rendered HTML
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# Now find the table
table = soup.find('table', id='serve-speed')
print(table.prettify())

driver.quit()

<table border="0" cellpadding="4" cellspacing="0" class="tablesorter" id="serve-speed">
 <thead>
  <tr>
   <th align="left">
    Match
   </th>
   <th align="left">
    Result
   </th>
   <th align="right" class="header">
    <span title="Average serve speed">
     Avg Speed
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average 1st serve speed">
     1st Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Standard deviation of
1st serve speed">
     1st StDev
    </span>
   </th>
   <th align="right" class="header">
    <span t""="" title="Average speed of 1st serves
down the ">
     1st T Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Average speed of
1st serves hit wide">
     1st Wide Avg
    </span>
   </th>
   <th align="right" class="header">
    <span title="Fastest second serve">
     Max 1st
    </span>
   </th>
   <th align="right" class="header">
    <span title="Slowest second serve">
  

In [10]:
speed = []

for row in table.find_all('tr'):
    # Extract the text from each cell in the row
    cells = row.find_all('td')
    if cells:  # Check if the row has any cells
        for cell in cells:
            print(cell.get_text(strip=True))
            speed.append((cell.get_text(strip=True)))


2021 Wimbledon QF
L vsHurkacz
107.0
112.7
6.5
117.4
109.1
122
88
95.6
4.8
99.2
95.4
102
85
2021 Wimbledon R16
W vsSonego
105.4
112.5
6.5
116.5
106.4
122
94
94.3
7.0
97.5
92.8
106
79
2021 Wimbledon R32
W vsNorrie
106.8
112.5
7.6
115.6
109.0
126
98
95.4
3.5
98.1
93.3
102
87
2021 Wimbledon R64
W vsGasquet
107.7
113.6
6.2
116.6
110.8
124
95
96.7
7.3
100.7
93.9
111
81
2021 Wimbledon R128
W vsMannarino
106.8
111.6
8.0
115.4
106.1
123
91
98.9
3.4
100.1
96.7
107
91
2019 US Open QF
L vsDimitrov
105.3
113.6
5.2
116.2
109.3
123
100
91.9
7.1
96.8
91.5
105
74
2019 US Open R16
W vsGoffin
108.4
113.0
7.0
116.6
108.8
125
95
96.9
4.3
101.0
94.5
106
91
2019 US Open R32
W vsEvans
106.2
113.1
7.4
118.6
108.3
124
94
92.4
7.4
96.6
92.0
104
81
2019 US Open R64
W vsDzumhur
108.2
113.7
6.2
116.3
109.9
123
96
94.0
6.2
97.2
91.2
108
82
2019 US Open R128
W vsNagal
105.7
112.9
7.0
118.3
107.8
123
93
93.0
6.5
93.6
95.4
105
79
2019 Wimbledon F
L vsDjokovic
109.2
116.6
6.3
120.4
113.2
126
99
95.8
5.2
95.5
98.4
108
85

In [11]:
speed = [s.replace('\xa0',' ') for s in speed]

In [12]:
year_tourney_round = speed[::15]

In [13]:
speed_df_rows = []
for i in range(0, len(speed)):
    if i % 15 == 0:
        row = speed[i:i+15]
        speed_df_rows.append(row)

In [14]:
cols = ['year tourney round', 'result opponent', 'avg speed', '1st: avg', '1st: stdev', '1st: t avg', '1st: wide avg', '1st: max', 
     '1st: min', '2nd: avg', '2nd: stdev', '2nd: t avg', '2nd: wide avg', '2nd: max', '2nd: min']

speed_df = pd.DataFrame(speed_df_rows, columns=cols)
speed_df['year'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[0])
speed_df['tourney'] = speed_df['year tourney round'].apply(lambda x: (' ').join(x.split(' ')[1:-1]))
speed_df['round'] = speed_df['year tourney round'].apply(lambda x: x.split(' ')[-1])
speed_df['result'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[0])
speed_df['opponent'] = speed_df['result opponent'].apply(lambda x: x.split(' ')[1][2:])
speed_df.drop(columns=['year tourney round', 'result opponent'], inplace=True)
speed_df.head(10)

,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min,year,tourney,round,result,opponent
0,107.0,112.7,6.5,117.4,109.1,122,88,95.6,4.8,99.2,95.4,102,85,2021,Wimbledon,QF,L,Hurkacz
1,105.4,112.5,6.5,116.5,106.4,122,94,94.3,7.0,97.5,92.8,106,79,2021,Wimbledon,R16,W,Sonego
2,106.8,112.5,7.6,115.6,109.0,126,98,95.4,3.5,98.1,93.3,102,87,2021,Wimbledon,R32,W,Norrie
3,107.7,113.6,6.2,116.6,110.8,124,95,96.7,7.3,100.7,93.9,111,81,2021,Wimbledon,R64,W,Gasquet
4,106.8,111.6,8.0,115.4,106.1,123,91,98.9,3.4,100.1,96.7,107,91,2021,Wimbledon,R128,W,Mannarino
5,105.3,113.6,5.2,116.2,109.3,123,100,91.9,7.1,96.8,91.5,105,74,2019,US Open,QF,L,Dimitrov
6,108.4,113.0,7.0,116.6,108.8,125,95,96.9,4.3,101.0,94.5,106,91,2019,US Open,R16,W,Goffin
7,106.2,113.1,7.4,118.6,108.3,124,94,92.4,7.4,96.6,92.0,104,81,2019,US Open,R32,W,Evans
8,108.2,113.7,6.2,116.3,109.9,123,96,94.0,6.2,97.2,91.2,108,82,2019,US Open,R64,W,Dzumhur
9,105.7,112.9,7.0,118.3,107.8,123,93,93.0,6.5,93.6,95.4,105,79,2019,US Open,R128,W,Nagal


In [15]:
neworder = ['year','tourney', 'round', 'result', 'opponent'] + cols[2:]
speed_df = speed_df[neworder]


In [16]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].replace('', np.nan)
speed_df

,year,tourney,round,result,opponent,avg speed,1st: avg,1st: stdev,1st: t avg,1st: wide avg,1st: max,1st: min,2nd: avg,2nd: stdev,2nd: t avg,2nd: wide avg,2nd: max,2nd: min
0,2021,Wimbledon,QF,L,Hurkacz,107.0,112.7,6.5,117.4,109.1,122,88,95.6,4.8,99.2,95.4,102,85
1,2021,Wimbledon,R16,W,Sonego,105.4,112.5,6.5,116.5,106.4,122,94,94.3,7.0,97.5,92.8,106,79
2,2021,Wimbledon,R32,W,Norrie,106.8,112.5,7.6,115.6,109.0,126,98,95.4,3.5,98.1,93.3,102,87
3,2021,Wimbledon,R64,W,Gasquet,107.7,113.6,6.2,116.6,110.8,124,95,96.7,7.3,100.7,93.9,111,81
4,2021,Wimbledon,R128,W,Mannarino,106.8,111.6,8.0,115.4,106.1,123,91,98.9,3.4,100.1,96.7,107,91
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166,2011,Australian Open,QF,W,Wawrinka,112.8,116.6,8.8,NaN,NaN,128,83,96.6,6.6,NaN,NaN,105,85
167,2011,Australian Open,R16,W,Robredo,107.4,114.2,8.3,NaN,NaN,126,80,92.5,6.6,NaN,NaN,102,78
168,2011,Australian Open,R32,W,Malisse,105.1,111.1,11.7,NaN,NaN,126,77,95.0,7.3,NaN,NaN,116,83
169,2011,Australian Open,R64,W,Simon,108.7,116.9,8.6,NaN,NaN,129,90,94.8,6.7,NaN,NaN,108,79


In [17]:
for col in speed_df.columns[5:]:
    speed_df[col] = speed_df[col].astype(float)

In [19]:
speed_df.to_csv('../data/federer_serve_speed.csv') #change the name of the file to save it as you want